# Trabalho prático 4: representação de dados em banco de dados relacional


## Criando a estrutura do banco


In [4]:
import sqlite3
import pandas as pd
caminho = './'
banco = 'banco.db'

conexao = sqlite3.connect(caminho + banco)

tabelas = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table'", conexao)

existe_tabela_paises = False
existe_tabela_indicadores = False
existe_tabela_dados = False

for i, j in tabelas.iterrows():
    if(j['name'] == 'paises'):
        existe_tabela_paises = True
    if(j['name'] == 'indicadores'):
        existe_tabela_indicadores = True
    if(j['name'] == 'dados'):
        existe_tabela_dados = True

if not(existe_tabela_paises):
    script_cria_tabela_pais = open(
        caminho + 'cria_tabela_pais.sql', 'r').read()
    conexao.executescript(script_cria_tabela_pais)

if not(existe_tabela_indicadores):
    script_cria_tabela_indicadores = open(
        caminho + 'cria_tabela_indicadores.sql', 'r').read()
    conexao.executescript(script_cria_tabela_indicadores)

if not(existe_tabela_dados):
    script_cria_tabela_dados = open(
        caminho + 'cria_tabela_dados.sql', 'r').read()
    conexao.executescript(script_cria_tabela_dados)

conexao.commit()
conexao.close()

## Inserindo dados na tabela países


In [7]:
import pandas as pd
import sqlite3

caminho: str = './'
arquivo: str = 'WDICountry.csv'
banco: str = 'banco.db'

conexao = sqlite3.connect(caminho + banco)

df_paises = pd.read_csv(caminho + arquivo, encoding = 'utf_8', encoding_errors= 'surrogateescape')

for coluna, linha in df_paises.iterrows():
    code = linha['Country Code']
    name = linha['Long Name']
    region = linha['Region']
    script = f"INSERT INTO paises (nome_curto, nome_longo, regiao) VALUES (\"{code}\",\"{name}\",\"{region}\");"
    conexao.executescript(script)

tabela_paises = pd.read_sql_query("SELECT * FROM paises", conexao)

conexao.close()

display(tabela_paises)


,pais_id,nome_curto,nome_longo,regiao
0,1,ABW,Aruba,Latin America & Caribbean
1,2,AFG,Islamic State of Afghanistan,South Asia
2,3,AGO,People's Republic of Angola,Sub-Saharan Africa
3,4,ALB,Republic of Albania,Europe & Central Asia
4,5,AND,Principality of Andorra,Europe & Central Asia
...,...,...,...,...
258,259,XKX,Republic of Kosovo,Europe & Central Asia
259,260,YEM,Republic of Yemen,Middle East & North Africa
260,261,ZAF,Republic of South Africa,Sub-Saharan Africa
261,262,ZMB,Republic of Zambia,Sub-Saharan Africa


## Inserindo dados na tabela indicadores

In [8]:
import pandas as pd
import sqlite3

caminho: str = './'
arquivo: str = 'WDIData.csv'
banco: str = 'banco.db'
conexao = sqlite3.connect(caminho + banco)

df_indicadores = pd.read_csv(
    caminho + arquivo, encoding='utf_8', encoding_errors='surrogateescape')


df_indicadores = df_indicadores[[
    'Indicator Name', 'Indicator Code']].drop_duplicates(subset=["Indicator Code"])

for coluna, linha in df_indicadores.iterrows():
    code = linha['Indicator Code']
    name = linha['Indicator Name']
    script = f"INSERT INTO indicadores (codigo, nome) VALUES (\"{code}\",\"{name}\");"
    conexao.executescript(script)

tabela_indicadores = pd.read_sql_query("SELECT * FROM indicadores", conexao)
conexao.close()

display(tabela_indicadores)


,indicador_id,codigo,nome
0,1,PA.NUS.PPP.05,"2005 PPP conversion factor, GDP (LCU per inter..."
1,2,PA.NUS.PRVT.PP.05,"2005 PPP conversion factor, private consumptio..."
2,3,EG.CFT.ACCS.ZS,Access to clean fuels and technologies for coo...
3,4,EG.ELC.ACCS.ZS,Access to electricity (% of population)
4,5,EG.ELC.ACCS.RU.ZS,"Access to electricity, rural (% of rural popul..."
...,...,...,...
1594,1595,SG.VAW.NEGL.ZS,Women who believe a husband is justified in be...
1595,1596,SG.VAW.REFU.ZS,Women who believe a husband is justified in be...
1596,1597,SP.M15.2024.FE.ZS,Women who were first married by age 15 (% of w...
1597,1598,SP.M18.2024.FE.ZS,Women who were first married by age 18 (% of w...


## Inserindo dados na tabela dados